In [9]:
pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain_experimental

Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install langchain_community

Note: you may need to restart the kernel to use updated packages.


### Built-in Tool - DuckDuckGo Search

In [28]:
from re import search
from unittest import result
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke("top news in india today")

print(results)

/home/hemil/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Check today's top 10 news stories, headline news, breaking news, latest news, politics news, sports news, entertainment news and business news on Times of India Check out the latest news from India and around the world. Latest India news on Bollywood, Politics, Business, Cricket, Technology and Travel. Aug 31, 2025 · Top News Stories of the day, Latest News Headlines, News Specials, Breaking News and Latest India News, World current affairs & Political News all around the clock at … Today's news: Get latest and Breaking News on Politics, Business, Lifestyle, Entertainment and Sports along with News updates from around the world. Also, find English News, live coverage … Times of India Covers all latest breaking news across India and world.


In [29]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


### Built-in Tool-Shell Tool

In [30]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('whoami')

print(results)

Executing command:
 whoami
hemil



/home/hemil/anaconda3/envs/langchain/lib/python3.10/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


### Custom Tools

In [31]:
from langchain_community.tools import tool

In [32]:
# Step 1 - create a function

def multiply(a, b):
    """Multiply two numbers"""
    return a*b

In [33]:
# Step 2 - add type hints

def multiply(a: int, b:int) ->int:
    """Multiply two numbers"""
    return a*b

In [34]:
# Step 3 - add tool decorator

@tool
def multiply(a:int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [35]:
result = multiply.invoke({"a":3, "b":5})

In [36]:
print(result)

15


In [37]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [38]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


### Method 2 -Using StructuredTool


In [39]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field


In [45]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

/home/hemil/anaconda3/envs/langchain/lib/python3.10/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(
/home/hemil/anaconda3/envs/langchain/lib/python3.10/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [42]:
def multiply_func(a: int, b: int) -> int:
    return a*b

In [46]:
from os import name


multiply_tool = StructuredTool.from_function(
    func = multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema = MultiplyInput
)

In [47]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


### Method 3 - Using BaseTool Class

In [48]:
from langchain.tools import BaseTool
from typing import Type

In [49]:
#args schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required= True, description = "The second number to add")

/home/hemil/anaconda3/envs/langchain/lib/python3.10/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(
/home/hemil/anaconda3/envs/langchain/lib/python3.10/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [50]:
class MultiplyTool(BaseTool):
    name: str ="multiply"
    description: str = "Multiply two numbers"
    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a:int ,b: int)-> int:
        return a*b

In [51]:
multiply_tool = MultiplyTool()

In [53]:
result= multiply_tool.invoke({'a': 3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


### Toolkit

In [54]:
from langchain_core.tools import tool

#Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two number"""
    return a+b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two number"""
    return a*b

In [55]:
class MathToolkit:
    def get_tools(self):
        return[add, multiply]

In [ ]:
toolkit = MathToolkit()me or dada jaa rahe hai lec me aaj 3 baje tere ko aana hai ?
for tool in tools:
    print(tool.name,"=>", tool.description)

add => Add two number
multiply => Multiply two number
